In [5]:
import requests
import json

agent_name = 'multi_tool_agent'  # Replace with your agent's name
user_id = 'u_testuser'
session_id = 's_testsession'

url = f"http://localhost:8000/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
headers = {
    "Content-Type": "application/json"
}
data = {
    "state": {}
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    print("Session creation successful:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error creating session: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")


Session creation successful:
{'id': 's_testsession', 'app_name': 'multi_tool_agent', 'user_id': 'u_testuser', 'state': {'state': {}}, 'events': [], 'last_update_time': 1747303729.910607}


In [6]:
agent_name = 'multi_tool_agent'  # Replace with your agent's name
user_id = 'u_testuser'
session_id = 's_testsession'
query_text = "weather in new york"

url = "http://localhost:8000/run"
headers = {
    "Content-Type": "application/json"
}
data = {
    "app_name": agent_name,
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {
        "role": "user",
        "parts": [{
            "text": query_text
        }]
    }
}

try:
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response.raise_for_status()
    print(f"\nQuery response for '{query_text}':")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error sending query: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Response content: {e.response.text}")


Query response for 'weather in new york':
[{'content': {'parts': [{'text': 'I am about to get the weather in new york, please wait.\n'}, {'functionCall': {'id': 'adk-75d55e79-4873-4cf8-9c81-865966ffa974', 'args': {'city': 'new york'}, 'name': 'get_weather'}}], 'role': 'model'}, 'invocation_id': 'e-3c19f165-241d-4ac2-931d-6a2ceb555620', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'I6mB5YFu', 'timestamp': 1747303735.960683}, {'content': {'parts': [{'functionResponse': {'id': 'adk-75d55e79-4873-4cf8-9c81-865966ffa974', 'name': 'get_weather', 'response': {'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit).'}}}], 'role': 'user'}, 'invocation_id': 'e-3c19f165-241d-4ac2-931d-6a2ceb555620', 'author': 'weather_time_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id':

In [11]:
query_text_sse = "weather in new york" # Example query
data_sse = {
    "app_name": agent_name,
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {
        "role": "user",
        "parts": [{
            "text": query_text_sse
        }]
    },
    "streaming": True # Enable token-level streaming
}
url_sse = "http://localhost:8000/run"

try:
    print(f"\nStreaming query response for '{query_text_sse}' (SSE enabled):")
    with requests.post(url_sse, headers=headers, data=json.dumps(data_sse), stream=True) as response:
        response.raise_for_status()
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith('data: '):
                    try:
                        event_data = json.loads(decoded_line[len('data: '):])
                        print(json.dumps(event_data, indent=2)) # Pretty print the JSON
                    except json.JSONDecodeError:
                        print(f"Could not decode JSON: {decoded_line[len('data: '):]}")
                elif decoded_line.strip(): # Print non-empty, non-data lines
                    print(decoded_line)
                # An empty line often signifies the end of an event or keep-alive ping

except requests.exceptions.RequestException as e:
    print(f"Error sending streaming SSE query: {e}")
    if hasattr(e, 'response') and e.response is not None:
        try:
            print(f"Response status code: {e.response.status_code}")
            print(f"Response content: {e.response.text}")
        except Exception as re:
            print(f"Further error reading response: {re}")



Streaming query response for 'weather in new york' (SSE enabled):
[{"content":{"parts":[{"text":"I am about to get the weather in new york, please wait.\n"},{"functionCall":{"id":"adk-9ec440d4-a2a5-433b-b399-a89b683f808c","args":{"city":"new york"},"name":"get_weather"}}],"role":"model"},"invocation_id":"e-a68db401-c379-4a60-abf4-154dd82c6a57","author":"weather_time_agent","actions":{"state_delta":{},"artifact_delta":{},"requested_auth_configs":{}},"long_running_tool_ids":[],"id":"TwCIleEA","timestamp":1747303926.913429},{"content":{"parts":[{"functionResponse":{"id":"adk-9ec440d4-a2a5-433b-b399-a89b683f808c","name":"get_weather","response":{"status":"success","report":"The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit)."}}}],"role":"user"},"invocation_id":"e-a68db401-c379-4a60-abf4-154dd82c6a57","author":"weather_time_agent","actions":{"state_delta":{},"artifact_delta":{},"requested_auth_configs":{}},"id":"lziQDbg5","timestamp":174730392